# LOAD MODEL

### IMPORTANT:
you may need to intstall the local version of your nnunetv2 first, instead of the version oion your docker

In [1]:
# !pip3 install -e /data/pathology/projects/pathology-lung-TIL/nnUNet_v2

# Libraries and functions

In [2]:
import os
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np

In [3]:
from nnunetv2.training.nnUNetTrainer.variants.pathology.nnUNetTrainer_custom_dataloader_test import nnUNetTrainer_custom_dataloader_test
from nnunetv2.utilities.file_path_utilities import load_json
import torch

nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
INSTEAD USING HARDCODED: /data/pathology/projects/pathology-lung-TIL/nnUNet_v2/data/nnUNet_raw
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
INSTEAD USING HARDCODED: /data/pathology/projects/pathology-lung-TIL/nnUNet_v2/data/nnUNet_preprocessed
nnUNet_results is not defined and nnU-Net cannot be used for training or inference. If this is not intended behavior, please read documentation/setting_up_paths.md for information on how to set this up.
INSTEAD USING HARDCODED: /data/pathology/projects/pathology-lung-TIL/nn

In [4]:
import time

In [5]:
current_os = "w" if os.name == "nt" else "l"
other_os = "l" if current_os == "w" else "w"

def convert_path(path, to=current_os):
    if to in ["w", "win", "windows"]:
        path = path.replace("/mnt/pa_cpg", "Y:")
        path = path.replace("/data/pathology", "Z:")
        path = path.replace("/mnt/pa_cpgarchive1", "W:")
        path = path.replace("/mnt/pa_cpgarchive2", "X:")
        path = path.replace("/", "\\")
    if to in ["u", "unix", "l", "linux"]:
        path = path.replace("Y:", "/mnt/pa_cpg")
        path = path.replace("Z:", "/data/pathology")
        path = path.replace("W:", "/mnt/pa_cpgarchive1")
        path = path.replace("X:", "/mnt/pa_cpgarchive2")
        path = path.replace("\\", "/")
    return path

In [6]:
labels = {
    "roi": 0, # in case 0 is true background instead of unannotated, make sure you add it as label 1, not 0. Since label 0 will not be used for counting
    "invasive tumor": 1,
    "tumor-associated stroma": 2,
    "in-situ tumor": 3 ,
    "healthy glands": 4,
    "necrosis not in-situ": 5,
    "inflamed stroma": 6,
    "rest": 7
}

colormap = [ [ 0, 0, 0, 0 ], [ 0, 224, 249, 255 ], [ 0, 249, 50, 255 ], [ 174, 249, 0, 255 ], [ 249, 100, 0, 255 ], [ 249, 0, 125, 255 ], [ 149, 0, 249, 255 ], [ 0, 0, 206, 255 ], [ 0, 185, 206, 255 ], [ 0, 206, 41, 255 ], [ 143, 206, 0, 255 ], [ 206, 82, 0, 255 ], [ 206, 0, 103, 255 ], [ 124, 0, 206, 255 ], [ 0, 0, 162, 255 ], [ 0, 145, 162, 255 ], [ 0, 162, 32, 255 ], [ 114, 162, 0, 255 ], [ 162, 65, 0, 255 ], [ 162, 0, 81, 255 ], [ 97, 0, 162, 255 ], [ 0, 0, 119, 255 ], [ 0, 107, 119, 255 ], [ 0, 119, 23, 255 ], [ 83, 119, 0, 255 ], [ 119, 47, 0, 255 ], [ 119, 0, 59, 255 ], [ 71, 0, 119, 255 ], [ 100, 100, 249, 255 ], [ 100, 234, 249, 255 ], [ 100, 249, 129, 255 ], [ 204, 249, 100, 255 ], [ 249, 159, 100, 255 ], [ 249, 100, 174, 255 ], [ 189, 100, 249, 255 ], [ 82, 82, 206, 255 ], [ 82, 193, 206, 255 ], [ 82, 206, 107, 255 ], [ 168, 206, 82, 255 ], [ 206, 131, 82, 255 ], [ 206, 82, 143, 255 ], [ 156, 82, 206, 255 ], [ 65, 65, 162, 255 ], [ 65, 152, 162, 255 ], [ 65, 162, 84, 255 ], [ 132, 162, 65, 255 ], [ 162, 104, 65, 255 ], [ 162, 65, 114, 255 ], [ 123, 65, 162, 255 ], [ 47, 47, 119, 255 ], [ 47, 112, 119, 255 ], [ 47, 119, 61, 255 ], [ 97, 119, 47, 255 ], [ 119, 76, 47, 255 ], [ 119, 47, 83, 255 ], [ 90, 47, 119, 255 ], [ 174, 174, 249, 255 ], [ 174, 242, 249, 255 ], [ 174, 249, 189, 255 ], [ 227, 249, 174, 255 ], [ 249, 204, 174, 255 ], [ 249, 174, 212, 255 ], [ 219, 174, 249, 255 ], [ 143, 143, 206, 255 ], [ 143, 199, 206, 255 ], [ 143, 206, 156, 255 ], [ 187, 206, 143, 255 ], [ 206, 168, 143, 255 ], [ 206, 143, 175, 255 ], [ 181, 143, 206, 255 ], [ 114, 114, 162, 255 ], [ 114, 157, 162, 255 ], [ 114, 162, 123, 255 ], [ 147, 162, 114, 255 ], [ 162, 132, 114, 255 ], [ 162, 114, 137, 255 ], [ 142, 114, 162, 255 ], [ 83, 83, 119, 255 ], [ 83, 115, 119, 255 ], [ 83, 119, 90, 255 ], [ 108, 119, 83, 255 ], [ 119, 97, 83, 255 ], [ 119, 83, 101, 255 ], [ 104, 83, 119, 255 ], [ 224, 224, 249, 255 ], [ 224, 247, 249, 255 ], [ 224, 249, 229, 255 ], [ 242, 249, 224, 255 ], [ 249, 234, 224, 255 ], [ 249, 224, 237, 255 ], [ 239, 224, 249, 255 ], [ 185, 185, 206, 255 ], [ 185, 204, 206, 255 ], [ 185, 206, 189, 255 ], [ 199, 206, 185, 255 ], [ 206, 193, 185, 255 ], [ 206, 185, 195, 255 ], [ 197, 185, 206, 255 ], [ 145, 145, 162, 255 ], [ 145, 160, 162, 255 ], [ 145, 162, 149, 255 ], [ 157, 162, 145, 255 ], [ 162, 152, 145, 255 ], [ 162, 145, 153, 255 ], [ 155, 145, 162, 255 ], [ 107, 107, 119, 255 ], [ 107, 118, 119, 255 ], [ 107, 119, 109, 255 ], [ 115, 119, 107, 255 ], [ 119, 112, 107, 255 ], [ 119, 107, 113, 255 ], [ 114, 107, 119, 255 ], [ 0, 0, 249, 255 ], [ 0, 224, 249, 255 ], [ 0, 249, 50, 255 ], [ 174, 249, 0, 255 ], [ 249, 100, 0, 255 ], [ 249, 0, 125, 255 ], [ 149, 0, 249, 255 ], [ 0, 0, 206, 255 ], [ 0, 185, 206, 255 ], [ 0, 206, 41, 255 ], [ 143, 206, 0, 255 ], [ 206, 82, 0, 255 ], [ 206, 0, 103, 255 ], [ 124, 0, 206, 255 ], [ 0, 0, 162, 255 ], [ 0, 145, 162, 255 ], [ 0, 162, 32, 255 ], [ 114, 162, 0, 255 ], [ 162, 65, 0, 255 ], [ 162, 0, 81, 255 ], [ 97, 0, 162, 255 ], [ 0, 0, 119, 255 ], [ 0, 107, 119, 255 ], [ 0, 119, 23, 255 ], [ 83, 119, 0, 255 ], [ 119, 47, 0, 255 ], [ 119, 0, 59, 255 ], [ 71, 0, 119, 255 ], [ 100, 100, 249, 255 ], [ 100, 234, 249, 255 ], [ 100, 249, 129, 255 ], [ 204, 249, 100, 255 ], [ 249, 159, 100, 255 ], [ 249, 100, 174, 255 ], [ 189, 100, 249, 255 ], [ 82, 82, 206, 255 ], [ 82, 193, 206, 255 ], [ 82, 206, 107, 255 ], [ 168, 206, 82, 255 ], [ 206, 131, 82, 255 ], [ 206, 82, 143, 255 ], [ 156, 82, 206, 255 ], [ 65, 65, 162, 255 ], [ 65, 152, 162, 255 ], [ 65, 162, 84, 255 ], [ 132, 162, 65, 255 ], [ 162, 104, 65, 255 ], [ 162, 65, 114, 255 ], [ 123, 65, 162, 255 ], [ 47, 47, 119, 255 ], [ 47, 112, 119, 255 ], [ 47, 119, 61, 255 ], [ 97, 119, 47, 255 ], [ 119, 76, 47, 255 ], [ 119, 47, 83, 255 ], [ 90, 47, 119, 255 ], [ 174, 174, 249, 255 ], [ 174, 242, 249, 255 ], [ 174, 249, 189, 255 ], [ 227, 249, 174, 255 ], [ 249, 204, 174, 255 ], [ 249, 174, 212, 255 ], [ 219, 174, 249, 255 ], [ 143, 143, 206, 255 ], [ 143, 199, 206, 255 ], [ 143, 206, 156, 255 ], [ 187, 206, 143, 255 ], [ 206, 168, 143, 255 ], [ 206, 143, 175, 255 ], [ 181, 143, 206, 255 ], [ 114, 114, 162, 255 ], [ 114, 157, 162, 255 ], [ 114, 162, 123, 255 ], [ 147, 162, 114, 255 ], [ 162, 132, 114, 255 ], [ 162, 114, 137, 255 ], [ 142, 114, 162, 255 ], [ 83, 83, 119, 255 ], [ 83, 115, 119, 255 ], [ 83, 119, 90, 255 ], [ 108, 119, 83, 255 ], [ 119, 97, 83, 255 ], [ 119, 83, 101, 255 ], [ 104, 83, 119, 255 ], [ 224, 224, 249, 255 ], [ 224, 247, 249, 255 ], [ 224, 249, 229, 255 ], [ 242, 249, 224, 255 ], [ 249, 234, 224, 255 ], [ 249, 224, 237, 255 ], [ 239, 224, 249, 255 ], [ 185, 185, 206, 255 ], [ 185, 204, 206, 255 ], [ 185, 206, 189, 255 ], [ 199, 206, 185, 255 ], [ 206, 193, 185, 255 ], [ 206, 185, 195, 255 ], [ 197, 185, 206, 255 ], [ 145, 145, 162, 255 ], [ 145, 160, 162, 255 ], [ 145, 162, 149, 255 ], [ 157, 162, 145, 255 ], [ 162, 152, 145, 255 ], [ 162, 145, 153, 255 ], [ 155, 145, 162, 255 ], [ 107, 107, 119, 255 ], [ 107, 118, 119, 255 ], [ 107, 119, 109, 255 ], [ 115, 119, 107, 255 ], [ 119, 112, 107, 255 ], [ 119, 107, 113, 255 ], [ 114, 107, 119, 255 ], [ 0, 0, 249, 255 ], [ 0, 224, 249, 255 ], [ 0, 249, 50, 255 ], [ 174, 249, 0, 255 ], [ 249, 100, 0, 255 ], [ 249, 0, 125, 255 ], [ 149, 0, 249, 255 ], [ 0, 0, 206, 255 ], [ 0, 185, 206, 255 ], [ 0, 206, 41, 255 ], [ 143, 206, 0, 255 ], [ 206, 82, 0, 255 ], [ 206, 0, 103, 255 ], [ 124, 0, 206, 255 ], [ 0, 0, 162, 255 ], [ 0, 145, 162, 255 ], [ 0, 162, 32, 255 ], [ 114, 162, 0, 255 ], [ 162, 65, 0, 255 ], [ 162, 0, 81, 255 ], [ 97, 0, 162, 255 ], [ 0, 0, 119, 255 ], [ 0, 107, 119, 255 ], [ 0, 119, 23, 255 ], [ 83, 119, 0, 255 ], [ 119, 47, 0, 255 ], [ 119, 0, 59, 255 ], [ 71, 0, 119, 255 ], [ 100, 100, 249, 255 ], [ 100, 234, 249, 255 ], [ 100, 249, 129, 255 ], [ 0, 249, 50, 255 ] ]
colormap = np.array(colormap)/255.
cmap = LinearSegmentedColormap.from_list('my_cmap', colors=colormap)

label_names = ['unannotated'] + list(labels.keys())[1:]
label_index = list(range(len(label_names)))
n_labels = len(label_names)
label_plot_args = {"cmap":cmap, "vmin":0, "vmax":255, "interpolation":"none"}

# Trainer initialization prep

### Set results folder

In [7]:
results_folder = convert_path(r'Z:\projects\pathology-lung-TIL\nnUNet_v2\data\nnUNet_results')

In [8]:
os.listdir(results_folder)

['Dataset004_TIGER_split', 'Dataset001_TIGER_split']

### Choose dataset

In [9]:
dataset = 'Dataset004_TIGER_split'

In [10]:
os.listdir(os.path.join(results_folder, dataset))

['nnUNetTrainer_custom_dataloader_test__nnUNetWholeSlideDataPlans__2d']

### Choose trainer__plans__config

In [11]:
trainer__plans__config = 'nnUNetTrainer_custom_dataloader_test__nnUNetWholeSlideDataPlans__2d'

In [12]:
trainer_results_folder = os.path.join(results_folder, dataset, trainer__plans__config)
trainer_results_folder

'/data/pathology/projects/pathology-lung-TIL/nnUNet_v2/data/nnUNet_results/Dataset004_TIGER_split/nnUNetTrainer_custom_dataloader_test__nnUNetWholeSlideDataPlans__2d'

In [13]:
sorted(os.listdir(trainer_results_folder))

['dataset.json',
 'fold_0',
 'fold_1',
 'fold_2',
 'fold_3',
 'fold_4',
 'plans.json',
 'splits.json',
 'train_config.json',
 'val_config.json']

### Choose fold

In [14]:
fold = 0

In [15]:
[file for file in os.listdir(os.path.join(trainer_results_folder, f'fold_{fold}')) if file.endswith('.pth')]

['checkpoint_final.pth', 'checkpoint_best.pth']

### Choose checkpoint

In [16]:
trainer_results_folder

'/data/pathology/projects/pathology-lung-TIL/nnUNet_v2/data/nnUNet_results/Dataset004_TIGER_split/nnUNetTrainer_custom_dataloader_test__nnUNetWholeSlideDataPlans__2d'

In [17]:
checkpoint_path = os.path.join(trainer_results_folder, f'fold_{fold}', 'checkpoint_best.pth')

### Get plans and dataset dicts

In [18]:
plans_dict = load_json(os.path.join(trainer_results_folder, 'plans.json'))
dataset_dict = load_json(os.path.join(trainer_results_folder, 'dataset.json'))

# Directly import class and initialize checkpoint

In [19]:
trainer = nnUNetTrainer_custom_dataloader_test(plans_dict, '2d', fold, dataset_dict)

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################



In [20]:
trainer.load_checkpoint(checkpoint_path)

In [24]:
trainer.current_epoch

948

In [23]:
dir(trainer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_best_ema',
 '_build_loss',
 '_get_deep_supervision_scales',
 '_save_debug_information',
 '_set_batch_size_and_oversample',
 'albumentations_aug',
 'aug',
 'batch_size',
 'build_network_architecture',
 'configuration_manager',
 'configuration_name',
 'configure_optimizers',
 'configure_rotation_dummyDA_mirroring_and_inital_patch_size',
 'current_epoch',
 'dataloader_train',
 'dataloader_val',
 'dataset_json',
 'device',
 'disable_checkpointing',
 'do_split',
 'fold',
 'folder_with_segs_from_previous_stage',
 'get_dataloaders',
 'get_plain_dataloaders',
 'get_tr_and_val_datasets',
 'get_training_transforms',
 'get_validat

# Get dataloaders

In [21]:
trainer.get_dataloaders.__code__.co_varnames[:trainer.get_dataloaders.__code__.co_argcount]

('self', 'subset', 'sample_double')

In [22]:
import inspect
# Get the signature of the function within the class
signature = inspect.signature(trainer.get_dataloaders)

# Get the parameters of the signature
parameters = signature.parameters

# Iterate over the parameters and print their names
for param in parameters.values():
    print(param.name)

subset
sample_double


In [23]:
tr_it, val_it = trainer.get_dataloaders(subset=True)





USING DATA SUBSET




Found splits.json
[Getting WSD dataloaders]
Using iterator template: /data/pathology/projects/pathology-lung-TIL/nnUNet_v2/nnunetv2/training/nnUNetTrainer/variants/pathology/wsd_iterator_alb_aug_template.json
Still timing everything, only copying SOME training and val files
[Creating batch iterators]
	[Creating TRAIN batch iterator]
	[Creating VAL batch iterator]
[Returning batch iterators]


### Get data

In [24]:
data = next(val_it)
data['data'] = data['data'][:2]
data['target'][0] = data['target'][0][:2]

In [25]:
%matplotlib inline

In [26]:
# # left is meant to be empty. its just easy to copy paste in figures this way
# fig, axs = plt.subplots(1, 4, figsize=(15, 4), gridspec_kw={'width_ratios': [10, 10, 10, 1], 'wspace': 0.5})
# axs[0].imshow(data['data'][0].permute((1, 2, 0)).cpu())
# axs[1].imshow(data['target'][0][0][0].cpu(), **label_plot_args)
# axs[2].imshow(data['data'][0].permute((1, 2, 0)).cpu())
# axs[2].imshow(data['target'][0][0][0].cpu(), alpha=0.3, **label_plot_args)
# axs[-1].imshow([[i] for i in list(range(len(label_names)))], **label_plot_args)
# axs[-1].set_yticks(label_index)
# axs[-1].set_yticklabels(label_names)
# axs[-1].yaxis.tick_right()
# axs[-1].get_xaxis().set_visible(False)
# axs[-1].set_title("Labels")
# plt.show()

# Prep Gaussian inference map and run sliding window on data

In [27]:
# data['data'].shape

In [28]:
# data['data'] = data['data'][:2]

In [29]:
# torch.cuda.empty_cache()
# out = torch.argmax(trainer.network(data['data'])[0][0], 0).cpu()

In [30]:
# plt.imshow(out, **label_plot_args)

### Run simple inference for multiple batches 

In [31]:
# for i in range(20):
#     data = next(val_it)
#     data['data'] = data['data'][:2, :, :512, :512]
#     data['target'][0] = data['target'][0][:2, :, :512, :512]

#     torch.cuda.empty_cache()
#     out = torch.argmax(trainer.network(data['data'])[0][0], 0).cpu()

#     # left is meant to be empty. its just easy to copy paste in figures this way
#     fig, axs = plt.subplots(1, 6, figsize=(15, 4), gridspec_kw={'width_ratios': [10, 10, 10, 10, 10, 1], 'wspace': 0.5})
#     axs[0].imshow(data['data'][0].permute((1, 2, 0)).cpu())
#     axs[1].imshow(data['target'][0][0][0].cpu(), **label_plot_args)
#     axs[2].imshow(data['data'][0].permute((1, 2, 0)).cpu())
#     axs[2].imshow(data['target'][0][0][0].cpu(), alpha=0.3, **label_plot_args)
#     axs[3].imshow(out, **label_plot_args)
#     axs[4].imshow(data['data'][0].permute((1, 2, 0)).cpu())
#     axs[4].imshow(out, alpha=0.3, **label_plot_args)
#     axs[-1].imshow([[i] for i in list(range(len(label_names)))], **label_plot_args)
#     axs[-1].set_yticks(label_index)
#     axs[-1].set_yticklabels(label_names)
#     axs[-1].yaxis.tick_right()
#     axs[-1].get_xaxis().set_visible(False)
#     axs[-1].set_title("Labels")
#     plt.show()

# Sliding window

In [32]:
tr_it, val_it = trainer.get_dataloaders(subset=True, sample_double=True)





USING DATA SUBSET




Found splits.json
[Getting WSD dataloaders]
Using iterator template: /data/pathology/projects/pathology-lung-TIL/nnUNet_v2/nnunetv2/training/nnUNetTrainer/variants/pathology/wsd_iterator_alb_aug_template.json
Still timing everything, only copying SOME training and val files
[Creating batch iterators]
	[Creating TRAIN batch iterator]
	[Creating VAL batch iterator]
[Returning batch iterators]


In [33]:
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor

In [34]:
from nnunetv2.paths import nnUNet_results, nnUNet_raw
import torch
from batchgenerators.utilities.file_and_folder_operations import join
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor

# instantiate the nnUNetPredictor
predictor = nnUNetPredictor(
    tile_step_size=0.5,
    use_gaussian=True,
    use_mirroring=True,
    perform_everything_on_gpu=True,
    device=torch.device('cuda', 0),
    verbose=False,
    verbose_preprocessing=False,
    allow_tqdm=True
)

In [35]:
# initializes the network architecture, loads the checkpoint
predictor.initialize_from_trained_model_folder(
    trainer_results_folder,
    use_folds=(0, 1, 2, 3, 4),
    checkpoint_name='checkpoint_best.pth',
)

In [36]:
batch = next(val_it)
batch['data'].shape

torch.Size([17, 3, 1024, 1024])

In [53]:
batch['data'].dtype

torch.float32

In [37]:
# pred_logits.shape

In [38]:
# batch['data'].permute((0, )).shape

BELOW: 7.83 s ± 598 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [39]:
# predictor.predict_from_data_iterator
# %timeit pred_logits = predictor.predict_logits_from_preprocessed_data(batch['data'].permute((1, 0, 2, 3)))

BELOW: 8.43 s ± 131 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [40]:
# %timeit pred_logits = predictor.get_logits_list_from_preprocessed_data(batch['data'].permute((1, 0, 2, 3)))

In [41]:
pred_logits = predictor.predict_logits_from_preprocessed_data(batch['data'].permute((1, 0, 2, 3)))
logits_list = predictor.get_logits_list_from_preprocessed_data(batch['data'].permute((1, 0, 2, 3)))

100%|██████████| 153/153 [00:01<00:00, 124.73it/s]


Prediction done, transferring to CPU if needed


100%|██████████| 153/153 [00:01<00:00, 123.98it/s]


Prediction done, transferring to CPU if needed


# reshape stuff

In [42]:
batch['data'].shape

torch.Size([17, 3, 1024, 1024])

In [43]:
# batch['data'].permute((1, 0, 2, 3))

# Function

In [44]:
def ensemble_softmax_list(x_batch):
    logits_list = predictor.get_logits_list_from_preprocessed_data(x_batch)
    softmax_list = [trainer.label_manager.apply_inference_nonlin(logits) for logits in logits_list]
    return softmax_list

In [45]:
softmax_list = ensemble_softmax_list(batch['data'].permute((1, 0, 2, 3)))

100%|██████████| 153/153 [00:01<00:00, 123.51it/s]


Prediction done, transferring to CPU if needed


In [46]:
# def ensemble_softmax_list(predictor, x_batch):
#     logits_list = predictor.get_logits_list_from_preprocessed_data(x_batch)
#     softmax_list = [trainer.apply_inference_nonlin(logits) for logits in logits_list]
#     return softmax_list

In [47]:
pred_seg = trainer.label_manager.convert_logits_to_segmentation(pred_logits)
pred_prob = trainer.label_manager.apply_inference_nonlin(pred_logits)

In [48]:
pred_prob.shape

torch.Size([8, 17, 1024, 1024])

In [49]:
pred_logits

tensor([[[[-3.5117e+00, -3.4102e+00, -3.4883e+00,  ..., -3.1055e+00,
           -3.1035e+00, -3.4004e+00],
          [-3.4160e+00, -3.2812e+00, -3.3555e+00,  ..., -2.9258e+00,
           -2.9219e+00, -3.2246e+00],
          [-3.4805e+00, -3.3496e+00, -3.4023e+00,  ..., -2.9688e+00,
           -2.9746e+00, -3.2598e+00],
          ...,
          [-3.4180e+00, -3.5000e+00, -3.5977e+00,  ..., -3.8281e+00,
           -3.6992e+00, -3.4375e+00],
          [-3.3008e+00, -3.3496e+00, -3.4570e+00,  ..., -3.6719e+00,
           -3.5098e+00, -3.2754e+00],
          [-3.4844e+00, -3.3906e+00, -3.4648e+00,  ..., -3.4902e+00,
           -3.3340e+00, -3.3184e+00]],

         [[-3.4434e+00, -3.2930e+00, -3.3652e+00,  ..., -3.4531e+00,
           -3.3594e+00, -3.4590e+00],
          [-3.2930e+00, -3.0938e+00, -3.1934e+00,  ..., -3.5039e+00,
           -3.3809e+00, -3.4121e+00],
          [-3.3633e+00, -3.1836e+00, -3.2695e+00,  ..., -3.6406e+00,
           -3.5312e+00, -3.5215e+00],
          ...,
     

In [50]:
pred_logits.shape

torch.Size([8, 17, 1024, 1024])

In [51]:
np.mean(ar, axis=0)

NameError: name 'ar' is not defined

In [ ]:
# softmax_fn = torch.nn.Softmax()

In [ ]:
pred_seg = trainer.label_manager.convert_logits_to_segmentation(pred_logits)

In [ ]:
for seg in pred_seg[:3]:
    plt.imshow(seg, **label_plot_args)
    plt.show()

In [ ]:
# plt.imshow(pred_seg)

In [ ]:
pred_logits

In [ ]:
# from nnunetv2.inference.export_prediction import export_prediction_from_logits, \
#     convert_predicted_logits_to_segmentation_with_correct_shape
# seg_fn2 = convert_predicted_logits_to_segmentation_with_correct_shape(pred_logits)

In [ ]:
pred_logits.shape

In [ ]:
%matplotlib inline

In [ ]:
1/0

In [ ]:
pred

In [ ]:
softmax_fn(pred).shape

In [ ]:
inference_gaussian = torch.from_numpy(
    compute_gaussian(
        trainer.configuration_manager.patch_size, 
        sigma_scale=1. / 8))

In [ ]:
pred = predict_sliding_window_return_logits(
    trainer.network, 
    data['data'], 
    trainer.label_manager.num_segmentation_heads,
    tile_size=trainer.configuration_manager.patch_size,
    mirror_axes=trainer.inference_allowed_mirroring_axes,
    tile_step_size=0.5,
    use_gaussian=True,
    precomputed_gaussian=inference_gaussian,
    perform_everything_on_gpu=True,
    verbose=True,
    device=trainer.device).cpu().numpy()

In [ ]:
prediction = predict_sliding_window_return_logits(self.network, data, num_seg_heads,
                                                                      tile_size=self.configuration_manager.patch_size,
                                                                      mirror_axes=self.inference_allowed_mirroring_axes,
                                                                      tile_step_size=0.5,
                                                                      use_gaussian=True,
                                                                      precomputed_gaussian=inference_gaussian,
                                                                      perform_everything_on_gpu=True,
                                                                      verbose=False,
                                                                      device=self.device).cpu().numpy()

In [ ]:
trainer.network

In [ ]:
# # RUN trainer EXPORT, doesnt work...

# trainer, plans, config = trainer__plans__config.split('__')
# folds = (0,)
# checkpoint = ('checkpoint_best.pth',)
# output_filename = 'trainer_export_test.zip'


# export_pretrained_trainer(dataset_name_or_id = dataset, 
#                         output_file = output_filename,
#                         configurations = ("2d", ),
#                         trainer = trainer,
#                         plans_identifier  = plans,
#                         folds = folds,
#                         strict = True,
#                         save_checkpoints = checkpoint,
#                         export_crossval_predictions = False)